## COVID 19 project : Week 1 Assignment

DATA SOURCES:

<div>
<img src="https://bryantarchway.com/wp-content/uploads/2020/04/corona.png" height="100" width="350"/>
</div>

World: https://www.worldometers.info/coronavirus/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/COVID-19_Outbreak_Cases_in_India.svg/1200px-COVID-19_Outbreak_Cases_in_India.svg.png" height="100" width="150"/>

India: https://www.mohfw.gov.in/

#### Importing Packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#### Initialising URLs for World and India

In [3]:
world_url = "https://www.worldometers.info/coronavirus/"
who_url = "https://www.who.int/emergencies/diseases/novel-coronavirus-2019"
corona_demographic_url = "https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/"
india_url = 'https://www.mohfw.gov.in/'

#### Validate Conectivity to the web-portals

In [4]:
def connectivity_checker(url):
    if requests.get(url).status_code == 200:
        return('Success!')
    elif requests.get(url).status_code == 404:
        return('Not Found.')
    
print("Connecting to Ministry of Health & Family Welfare Portal...")
print("Status: ",connectivity_checker(india_url))
print("Connecting to Worldometers Portal...")
print("Status: ",connectivity_checker(world_url))
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))
print("Connecting to WHO Portal...")
print("Status: ",connectivity_checker(who_url))

Connecting to Ministry of Health & Family Welfare Portal...
Status:  Success!
Connecting to Worldometers Portal...
Status:  Success!
Connecting to Worldometers Corona Demographics Portal...
Status:  Success!
Connecting to WHO Portal...
Status:  Success!


### WEB SCRAPPING INDIA DATA
<div>
<img src="https://surroundvision.co.uk/wp-content/uploads/2018/05/Portfolio-Incredible-India-Banner-1920x900.jpg" height="100" width="550"/>
</div>

#### Scrapping Data

In [47]:
soup=BeautifulSoup(requests.get(india_url).content, "html.parser")
div_container = soup.find_all("div",class_="data-table table-responsive")
data_table_body = div_container[0].table.tbody    
tag = data_table_body.findAll('td')

In [48]:
confirmed_list = []
cured_list = []
deaths_list = []
states_list = []

def data_extractor(data_list,llimit,interval):
    for data_cntr in range(llimit,160,interval): data_list.append(tag[data_cntr].text) 
    return data_list

states = data_extractor(states_list,1,5)
confirmed_cases = data_extractor(confirmed_list,2,5)
cured_cases = data_extractor(cured_list,3,5)
death_cases = data_extractor(deaths_list,4,5)


#### Exporting data into dataframe:

In [50]:
df_india = pd.DataFrame(list(zip(states, confirmed_cases,cured_cases,death_cases)), columns=["States","Confirmed_Cases","Cured_Cases","Death_Cases"])
df_india.head(5)

,States,Confirmed_Cases,Cured_Cases,Death_Cases
0,Andaman and Nicobar Islands,33,16,0
1,Andhra Pradesh,1463,403,33
2,Arunachal Pradesh,1,1,0
3,Assam,42,29,1
4,Bihar,426,82,2


#### Data Cleaning

In [515]:
def missing_val_updater(dfname):
    """
    Replacing missing values with Zero
    """
    dfname.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    dfname.replace(np.nan,0, inplace=True) 
    return dfname

def detect_missing_val(df):
    """
    Detecting missing value
    """
    tot_col = len(df.columns)
    for i in range(1,tot_col):
        if df[df[df.columns[i]].isna()][df.columns[i]].shape[0] == 0:
            missing_val_updater(df)
            print("Replaced missing values with Zero for :",df.columns[i])
        else:
            print("No Missing Values for :",df.columns[i])

detect_missing_val(df_india)

Replaced missing values with Zero for : Confirmed_Cases
Replaced missing values with Zero for : Cured_Cases
Replaced missing values with Zero for : Death_Cases


### WEB SCRAPPING WORLD DATA

<div>
<img src="http://www.pngmart.com/files/11/Earth-Travel-World-PNG-Transparent-Image.png" height="100" width="750"/>
</div>

#### Scrapping Data

In [57]:
soup=BeautifulSoup(requests.get(world_url).content, "html.parser")

In [58]:
coronatable=soup.find_all("table")
co=coronatable[0]
country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,critical=[],[],[],[],[],[],[],[]
rows=co.find_all("tr")[9:-4]
rows
for row in rows:
    col=row.find_all("td")
    country.append(col[0].text.strip())
    total_cases.append(col[1].text.strip())
    new_cases.append(col[2].text.strip())
    total_deaths.append(col[3].text)
    new_deaths.append(col[4].text)
    total_recovered.append(col[5].text)
    active_cases.append(col[6].text)
    critical.append(col[7].text)

#### Exporting raw data into dataframe:

In [59]:
raw_df=pd.DataFrame(list(zip(country, total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,critical)), columns=["Country","Tot_Cases","New_Cases","Tot_Deaths","New_Deaths","Tot_Recovered","Active","Critical"])
raw_df.head()

,Country,Tot_Cases,New_Cases,Tot_Deaths,New_Deaths,Tot_Recovered,Active,Critical
0,USA,"1,182,615","+21,841","68,247",+803,"166,203","948,165","16,156"
1,Spain,"247,122","+1,533","25,264",+164,"148,558","73,300","2,386"
2,Italy,"210,717","+1,389","28,884",+174,"81,654","100,179","1,501"
3,UK,"186,599","+4,339","28,446",+315,N/A,"157,809","1,559"
4,France,"168,693",+297,"24,895",+135,"50,784","93,014","3,819"


#### Data Cleaning

In [60]:
def missing_val_updater(dfname):
    """
    Replacing missing values with Zero
    """
    dfname.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    dfname.replace(np.nan,0, inplace=True)
    return dfname

def records_cleaner(val):
    """
    Converting strings to integers for numeric manipulation. 
    Removing symbols "+" and ","
    """
    records_list=[]
    for i in df_world[val]:
        s=str(i)
        s1=s.replace("+","").replace("N/A","0")
        s2=s1.replace(",","")
        s3=int(s2)
        records_list.append(s3)
    return records_list


df_world = missing_val_updater(raw_df)

u_total_cases = records_cleaner("Tot_Cases")
u_new_cases = records_cleaner("New_Cases")
u_total_deaths = records_cleaner("Tot_Deaths")
u_new_deaths = records_cleaner("New_Deaths")
u_total_recovered = records_cleaner("Tot_Recovered")
u_active_cases = records_cleaner("Active")
u_critical = records_cleaner("Critical")

df_world=pd.DataFrame(list(zip(country, u_total_cases,u_new_cases,u_total_deaths,u_new_deaths,u_total_recovered,u_active_cases,critical)), columns=["Country","Tot_Cases","New_Cases","Tot_Deaths","New_Deaths","Tot_Recovered","Active","Critical"])
df_world.head()


,Country,Tot_Cases,New_Cases,Tot_Deaths,New_Deaths,Tot_Recovered,Active,Critical
0,USA,1182615,21841,68247,803,166203,948165,"16,156"
1,Spain,247122,1533,25264,164,148558,73300,"2,386"
2,Italy,210717,1389,28884,174,81654,100179,"1,501"
3,UK,186599,4339,28446,315,0,157809,"1,559"
4,France,168693,297,24895,135,50784,93014,"3,819"


#### Demographics : Segmentation based on Age and types of medical conditions

In [51]:
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))

soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[0]
tr_tag = table_body.findAll('tr')
deaths_list = []
share_deaths_list = []
age_list = []
pre_ex_cond_list = []
w_pre_ex_cond_list = []
unknwn_pre_ex_cond_list = []
total = []

for i in range(1,6): age_list.append(tr_tag[i].td.div.text)
limit = len(age_list)
for x in range(0,limit):
    if x == limit-1:
        age_list[x] = age_list[x].replace("years old", "").replace("+","").strip()
        age_list[x] = age_list[x]+' - 104'
    else:    
        age_list[x] = age_list[x].replace("years old", "").strip()   
for y in range(1,limit+1):
    td_tag = tr_tag[y].findAll("td")
    deaths_list.append(td_tag[1].text)
    share_deaths_list.append(td_tag[2].text)
    pre_ex_cond_list.append(td_tag[3].text)
    w_pre_ex_cond_list.append(td_tag[4].text)
    unknwn_pre_ex_cond_list.append(td_tag[5].text)

def data_cleaner(data_list,dtype):
    for x in range(0,len(data_list)): data_list[x] = dtype(data_list[x].replace(",","").replace("%","").replace("+",""))
    return data_list


u_death_list = data_cleaner(deaths_list,int)
u_share_deaths_list = data_cleaner(share_deaths_list,float)
u_pre_ex_cond_list = data_cleaner(pre_ex_cond_list,int)
u_w_pre_ex_cond_list = data_cleaner(w_pre_ex_cond_list,int)
u_unknwn_pre_ex_cond_list = data_cleaner(unknwn_pre_ex_cond_list,int)


agewise_df = pd.DataFrame(list(zip(age_list,u_death_list, u_share_deaths_list,u_pre_ex_cond_list,u_w_pre_ex_cond_list,u_unknwn_pre_ex_cond_list)), columns=["Age Groups","Deaths","Share of Deaths(%)","PC","WPC","UPC"])
agewise_df


Connecting to Worldometers Corona Demographics Portal...
Status:  Success!


,Age Groups,Deaths,Share of Deaths(%),PC,WPC,UPC
0,0 - 17,3,0.04,3,0,0
1,18 - 44,309,4.50,244,25,40
2,45 - 64,1581,23.10,1343,59,179
3,65 - 74,1683,24.60,1272,26,385
4,75 - 104,3263,47.70,2289,27,947


#### TOTAL: Age-wise Segmentation

In [430]:
tot_tag = tr_tag[6].findAll("td")
u_total_deaths = int(tot_tag[1].text.replace(",","").replace("%",""))
u_share_of_deaths = int(tot_tag[2].text.replace(",","").replace("%",""))
u_TPC = int(tot_tag[3].text.replace(",","").replace("%",""))
u_TWPC = int(tot_tag[4].text.replace(",","").replace("%",""))
u_TUPC = int(tot_tag[5].text.replace(",","").replace("%",""))

total_agewise_df = pd.DataFrame([[u_total_deaths, u_share_of_deaths,u_TPC,u_TWPC,u_TUPC]], columns=["Total Deaths","Total Share of Deaths(%)","TPC","TWPC","TUPC"])
total_agewise_df


,Total Deaths,Total Share of Deaths(%),TPC,TWPC,TUPC
0,6839,100,5151,137,1551


#### Demographics : Segmentation based on Gender and types of medical conditions

In [34]:
print("Connecting to Worldometers Corona Demographics Portal...")
print("Status: ",connectivity_checker(corona_demographic_url))

soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[1]
table_body
tr_tag = table_body.findAll('tr')
tr_tag
deaths_list = []
share_deaths_list,share_pre_ex_cond_list,share_w_pre_ex_cond_list,share_unknwn_pre_ex_cond_list = [],[],[],[]
gender_list = []
pre_ex_cond_list = []
w_pre_ex_cond_list = []
unknwn_pre_ex_cond_list = []


for i in range(1,3): gender_list.append(tr_tag[i].td.div.text)

limit = len(gender_list)
 
for y in range(1,limit+1):
    td_tag = tr_tag[y].findAll("td")
    deaths_list.append(td_tag[1].text)
    share_deaths_list.append(td_tag[2].text)
    pre_ex_cond_list.append(td_tag[3].text)
    share_pre_ex_cond_list.append(td_tag[4].text)
    w_pre_ex_cond_list.append(td_tag[5].text)
    share_w_pre_ex_cond_list.append(td_tag[6].text)
    unknwn_pre_ex_cond_list.append(td_tag[7].text)
    share_unknwn_pre_ex_cond_list.append(td_tag[8].text)

def data_cleaner(data_list,dtype):
    if data_list == pre_ex_cond_list or data_list == w_pre_ex_cond_list or data_list == unknwn_pre_ex_cond_list :
        for x in range(0,len(data_list)): data_list[x] = data_list[x].replace(".","").replace(",","").replace("%","")
            
    elif data_list == share_w_pre_ex_cond_list or data_list == share_pre_ex_cond_list or data_list == share_deaths_list:
        for x in range(0,len(data_list)): data_list[x] = dtype(data_list[x].replace(",",".").replace("%",""))
            
    else:
        for x in range(0,len(data_list)): data_list[x] = dtype(data_list[x].replace(",","").replace("%",""))
            
    return data_list


u_death_list = data_cleaner(deaths_list,int)
u_share_deaths_list = data_cleaner(share_deaths_list,float)
u_pre_ex_cond_list = data_cleaner(pre_ex_cond_list,float)

u_w_pre_ex_cond_list = data_cleaner(w_pre_ex_cond_list,float)
u_unknwn_pre_ex_cond_list = data_cleaner(unknwn_pre_ex_cond_list,float)
u_share_unknwn_pre_ex_cond_list = data_cleaner(share_unknwn_pre_ex_cond_list,float)
u_share_w_pre_ex_cond_list = data_cleaner(share_w_pre_ex_cond_list,float)
u_share_pre_ex_cond_list = data_cleaner(share_pre_ex_cond_list,float)




genderwise_df = pd.DataFrame(list(zip(gender_list,u_death_list, u_share_deaths_list,u_pre_ex_cond_list,u_share_pre_ex_cond_list,u_w_pre_ex_cond_list,u_share_w_pre_ex_cond_list,u_unknwn_pre_ex_cond_list,u_share_unknwn_pre_ex_cond_list)), columns=["Gender","Deaths","Share_of_Deaths(%)","PC","Share_of_PC(%)","WPC","Share_of_WPC(%)","UPC","Share_of_UPC(%)"])
genderwise_df


Connecting to Worldometers Corona Demographics Portal...
Status:  Success!


,Gender,Deaths,Share_of_Deaths(%),PC,Share_of_PC(%),WPC,Share_of_WPC(%),UPC,Share_of_UPC(%)
0,Male,4095,61.8,3087,62.2,96,72.2,912,59.5
1,Female,2530,38.2,1873,37.8,37,27.8,620,40.5


#### Demographics : Probability of Death by Age

In [498]:
soup=BeautifulSoup(requests.get(corona_demographic_url).content, "html.parser")
div_container = soup.find_all("table",class_="table table-hover table-bordered table-condensed table-list")
table_body = div_container[2]
table_body
tr_tag = table_body.findAll('tr')

age_list = []
death_rate_confirmed_list = []
death_rate_all_cases_list = []

limit = len(tr_tag)
for i in range(1,limit): age_list.append(tr_tag[i].td.text)
 
for y in range(1,limit):
    td_tag = tr_tag[y].findAll("td")
    death_rate_confirmed_list.append(td_tag[1].text)
    death_rate_all_cases_list.append(td_tag[2].text)

deathrate_df = pd.DataFrame(list(zip(age_list,death_rate_confirmed_list, death_rate_all_cases_list)), columns=["Age","Death Rate (Confirmed)","Death Rate (All Cases)"])
deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80+ years old,21.9%,14.8%
1,70-79 years old,,8.0%
2,60-69 years old,,3.6%
3,50-59 years old,,1.3%
4,40-49 years old,,0.4%
5,30-39 years old,,0.2%
6,20-29 years old,,0.2%
7,10-19 years old,,0.2%
8,0-9 years old,,no fatalities


In [511]:
def detect_missing_val(df):
    tot_col = len(df.columns)
    print(tot_col)
    for i in range(1,tot_col):
        if df[df[df.columns[i]].isna()][df.columns[i]].shape[0] == 0:
            print("Missing Values detected for column : ",df.columns[i])
            missing_val_updater(df)
            print("Replaced missing values with Zero")
        else:
            print("No Missing Values")

detect_missing_val(deathrate_df)

3
Missing Values detected for column :  Death Rate (Confirmed)
Replaced missing values with Zero
Missing Values detected for column :  Death Rate (All Cases)
Replaced missing values with Zero


In [513]:
deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80+ years old,21.9%,14.8%
1,70-79 years old,0,8.0%
2,60-69 years old,0,3.6%
3,50-59 years old,0,1.3%
4,40-49 years old,0,0.4%
5,30-39 years old,0,0.2%
6,20-29 years old,0,0.2%
7,10-19 years old,0,0.2%
8,0-9 years old,0,no fatalities


In [522]:
def records_cleaner(val,dtype):
    """
    Converting strings to integers for numeric manipulation. 
    Removing symbols and redundant strings
    """
    records_list=[]
    for i in deathrate_df[val]:
        s=str(i)
        s1=dtype(s.replace("+","").replace(",","").replace("%","").replace("years old", "").replace("no fatalities","0.0").strip())
        records_list.append(s1)
    return records_list

updated_age = records_cleaner("Age",str)
updated_death_rate_cl = records_cleaner("Death Rate (Confirmed)",float)
updated_death_rate_acl = records_cleaner("Death Rate (All Cases)",float)
updated_deathrate_df = pd.DataFrame(list(zip(updated_age,updated_death_rate_cl, updated_death_rate_acl)), columns=["Age","Death Rate (Confirmed)","Death Rate (All Cases)"])
 

updated_deathrate_df


,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80,21.9,14.8
1,70-79,0.0,8.0
2,60-69,0.0,3.6
3,50-59,0.0,1.3
4,40-49,0.0,0.4
5,30-39,0.0,0.2
6,20-29,0.0,0.2
7,10-19,0.0,0.2
8,0-9,0.0,0.0


#### TIME SERIES DATA
##### Source: Johns Hopkins CSSE

In [536]:
ts_confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
ts_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30


#### Data Cleaning 
#### Replacing Missing Values or NaN with Zero

In [542]:
detect_missing_val(ts_confirmed_df)

Replaced missing values with Zero for : Country/Region
Replaced missing values with Zero for : Lat
Replaced missing values with Zero for : Long
Replaced missing values with Zero for : 1/22/20
Replaced missing values with Zero for : 1/23/20
Replaced missing values with Zero for : 1/24/20
Replaced missing values with Zero for : 1/25/20
Replaced missing values with Zero for : 1/26/20
Replaced missing values with Zero for : 1/27/20
Replaced missing values with Zero for : 1/28/20
Replaced missing values with Zero for : 1/29/20
Replaced missing values with Zero for : 1/30/20
Replaced missing values with Zero for : 1/31/20
Replaced missing values with Zero for : 2/1/20
Replaced missing values with Zero for : 2/2/20
Replaced missing values with Zero for : 2/3/20
Replaced missing values with Zero for : 2/4/20
Replaced missing values with Zero for : 2/5/20
Replaced missing values with Zero for : 2/6/20
Replaced missing values with Zero for : 2/7/20
Replaced missing values with Zero for : 2/8/20


In [541]:
ts_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,0,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,0,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,0,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,0,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,0,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30


#### CONTINENT-WISE DATA

In [62]:
"""
Continent-wise Segregation
"""

soup=BeautifulSoup(requests.get(world_url).content, "html.parser")
tr_tags = soup.find_all('tr')
continent_list = []
total_cases = []
new_cases = []
total_deaths = []
new_deaths = []
total_recovered = []
active_cases,critical=[],[]

def missing_val_updater(dfname):
    """
    Replacing missing values with Zero
    """
    dfname.replace(r'^\s*$', np.nan, regex=True, inplace=True)
    dfname.replace(np.nan,0, inplace=True)
    return dfname

for i in range(1,7): 
    continent_list.append(tr_tags[i].td.text.replace("\n",""))
    td_tags = tr_tags[i].findAll('td')
    total_cases.append(td_tags[1].text)
    new_cases.append(td_tags[2].text)
    total_deaths.append(td_tags[3].text)
    new_deaths.append(td_tags[4].text)
    total_recovered.append(td_tags[5].text)
    active_cases.append(td_tags[6].text)
    critical.append(td_tags[7].text)
    
raw_continent_df = pd.DataFrame(list(zip(continent_list, total_cases,new_cases,total_deaths,new_deaths,total_recovered,active_cases,critical)), columns=["Continent_List","Tot_Cases","New_Cases","Tot_Deaths","New_Deaths","Tot_Recovered","Active","Critical"])
print("------[C O N T I N E N T S] Raw Data ------")
print(raw_continent_df)


updated_continent_df = missing_val_updater(raw_continent_df)
updated_continent_df

def records_cleaner(val):
    """
    Converting strings to integers for numeric manipulation. 
    Removing symbols "+" and ","
    """
    records_list=[]
    for i in updated_continent_df[val]:
        s=str(i)
        s1=s.replace("+","")
        s2=s1.replace(",","")
        s3=int(s2)
        records_list.append(s3)
    return records_list

updated_total_cases = records_cleaner("Tot_Cases")
updated_new_cases = records_cleaner("New_Cases")
updated_total_deaths = records_cleaner("Tot_Deaths")
updated_new_deaths = records_cleaner("New_Deaths")
updated_total_recovered = records_cleaner("Tot_Recovered")
u_active_cases = records_cleaner("Active")
u_critical = records_cleaner("Critical")


updated_continent_df = pd.DataFrame(list(zip(continent_list, updated_total_cases,updated_new_cases,updated_total_deaths,updated_new_deaths,updated_total_recovered,u_active_cases,u_critical)), columns=["Continent_List","Tot_Cases","New_Cases","Tot_Deaths","New_Deaths","Tot_Recovered","Active","Critical"])
print("\n------[C O N T I N E N T S] Cleaned Data ------")
print(updated_continent_df)

#sorting continents by increasing of deaths
sorted_df = updated_continent_df.sort_values(by=['Tot_Deaths'],ascending=False)
print("\n------[C O N T I N E N T S] Sorted by Total_Deaths ------")
print(sorted_df)

#continent with the highest death count and continent with lowest death count
max_deaths_obj = sorted_df.loc[sorted_df['Tot_Deaths'].idxmax()]
min_deaths_obj = sorted_df.loc[sorted_df['Tot_Deaths'].idxmin()]

print('\nContinent with Highest Deaths: ',max_deaths_obj['Continent_List'])
print('Total_Deaths: ',max_deaths_obj['Tot_Deaths'])
print('\nContinent with Lowest Deaths: ',min_deaths_obj['Continent_List'])
print('Total_Deaths: ',min_deaths_obj['Tot_Deaths'])

------[C O N T I N E N T S] Raw Data ------
  Continent_List  Tot_Cases New_Cases Tot_Deaths New_Deaths Tot_Recovered  \
0  North America  1,285,455   +26,853     74,796     +1,016       220,924   
1         Europe  1,447,189   +22,803    140,573     +1,163       540,667   
2           Asia    558,461   +12,651     19,602       +263       294,200   
3  South America    208,094    +8,344     10,571       +328        71,856   
4        Oceania      8,390       +22        115         +2         7,165   
5         Africa     45,003    +1,269      1,794        +31        15,040   

    Active Critical  
0  989,735   17,403  
1  765,949   17,271  
2  244,659    5,366  
3  125,667    9,853  
4    1,110       31  
5   28,169      130  

------[C O N T I N E N T S] Cleaned Data ------
  Continent_List  Tot_Cases  New_Cases  Tot_Deaths  New_Deaths  Tot_Recovered  \
0  North America    1285455      26853       74796        1016         220924   
1         Europe    1447189      22803      140573 

# Consolidated Findings

#### Age-wise Coronavirus Deaths [NEW YORK]
Source : [New York City Health](https://www1.nyc.gov/assets/doh/downloads/pdf/imm/covid-19-daily-data-summary-deaths-04152020-1.pdf)

Website : https://www.worldometers.info/coronavirus/coronavirus-age-sex-demographics/

- Death Rate = (number of deaths / number of cases) = probability of dying if infected by the virus (%)

In [523]:
updated_deathrate_df

,Age,Death Rate (Confirmed),Death Rate (All Cases)
0,80,21.9,14.8
1,70-79,0.0,8.0
2,60-69,0.0,3.6
3,50-59,0.0,1.3
4,40-49,0.0,0.4
5,30-39,0.0,0.2
6,20-29,0.0,0.2
7,10-19,0.0,0.2
8,0-9,0.0,0.0


#### Genderwise Segmentation [NEW YORK]
- PC  : Pre-existing Conditions
- WPC : Without Pre-existing Conditions
- UPC : Unknown Pre-existing Conditions

In [35]:
genderwise_df

,Gender,Deaths,Share_of_Deaths(%),PC,Share_of_PC(%),WPC,Share_of_WPC(%),UPC,Share_of_UPC(%)
0,Male,4095,61.8,3087,62.2,96,72.2,912,59.5
1,Female,2530,38.2,1873,37.8,37,27.8,620,40.5


#### Agewise Segmentation [NEW YORK]
- PC  : Pre-existing Conditions
- WPC : Without Pre-existing Conditions
- UPC : Unknown Pre-existing Conditions


Upper Limit for Age Group set to 104

Oldest age for Corona Survivor in New York, A World War II Veteran

Source: https://www.straitstimes.com/world/united-states/104-year-old-american-is-worlds-oldest-coronavirus-survivor

In [52]:
agewise_df

,Age Groups,Deaths,Share of Deaths(%),PC,WPC,UPC
0,0 - 17,3,0.04,3,0,0
1,18 - 44,309,4.50,244,25,40
2,45 - 64,1581,23.10,1343,59,179
3,65 - 74,1683,24.60,1272,26,385
4,75 - 104,3263,47.70,2289,27,947


#### Total Agewise Segmentation [NEW YORK]
- TPC  : Total Pre-existing Conditions
- TWPC : Total Without Pre-existing Conditions
- TUPC : Total Unknown Pre-existing Conditions

In [527]:
total_agewise_df

,Total Deaths,Total Share of Deaths(%),TPC,TWPC,TUPC
0,6839,100,5151,137,1551


#### Corona Cases by Continent

In [63]:
print('\nContinent with Highest Deaths: ',max_deaths_obj['Continent_List'])
print('Total_Deaths: ',max_deaths_obj['Tot_Deaths'])
print('\nContinent with Lowest Deaths: ',min_deaths_obj['Continent_List'])
print('Total_Deaths: ',min_deaths_obj['Tot_Deaths'])
updated_continent_df


Continent with Highest Deaths:  Europe
Total_Deaths:  140573

Continent with Lowest Deaths:  Oceania
Total_Deaths:  115


,Continent_List,Tot_Cases,New_Cases,Tot_Deaths,New_Deaths,Tot_Recovered,Active,Critical
0,North America,1285455,26853,74796,1016,220924,989735,17403
1,Europe,1447189,22803,140573,1163,540667,765949,17271
2,Asia,558461,12651,19602,263,294200,244659,5366
3,South America,208094,8344,10571,328,71856,125667,9853
4,Oceania,8390,22,115,2,7165,1110,31
5,Africa,45003,1269,1794,31,15040,28169,130


#### Corona Cases by Country

In [61]:
df_world.head(7)

,Country,Tot_Cases,New_Cases,Tot_Deaths,New_Deaths,Tot_Recovered,Active,Critical
0,USA,1182615,21841,68247,803,166203,948165,"16,156"
1,Spain,247122,1533,25264,164,148558,73300,"2,386"
2,Italy,210717,1389,28884,174,81654,100179,"1,501"
3,UK,186599,4339,28446,315,0,157809,"1,559"
4,France,168693,297,24895,135,50784,93014,"3,819"
5,Germany,165383,416,6840,28,130600,27943,"1,979"
6,Russia,134687,10633,1280,58,16639,116768,"2,300"


#### Corona Cases by State [INDIA]

In [529]:
df_india.head()

,States,Confirmed_Cases,Cured_Cases,Death_Cases
0,Andaman and Nicobar Islands,33,16,0
1,Andhra Pradesh,1463,403,33
2,Arunachal Pradesh,1,1,0
3,Assam,42,29,1
4,Bihar,426,82,2


#### Time Series records for Countrywise Corona Cases

In [539]:
ts_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20
0,0,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,1176,1279,1351,1463,1531,1703,1828,1939,2171,2335
1,0,Albania,41.1533,20.1683,0,0,0,0,0,0,...,634,663,678,712,726,736,750,766,773,782
2,0,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,2910,3007,3127,3256,3382,3517,3649,3848,4006,4154
3,0,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,723,723,731,738,738,743,743,743,745,745
4,0,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,25,25,25,25,26,27,27,27,27,30
